# Petitions Classification


In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os

os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Tutorials')
os.chdir('DeepLearningProject')
os.chdir('PetitionClassification')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import time

In [4]:
df = pd.read_csv('data/crawling.csv')
df.head()

,category,content,count,end,start,title
0,인권/성평등,본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 ...,267,2020-02-01,2020-01-02,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...
1,경제민주화,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...,271,2020-02-01,2020-01-02,주식시장 활성화 및 소액(개미)투자자 보호
2,행정,억울한 일로 국민청원을 신청합니다.\r\n\r\r\n 저는 **구치소 **교도관입니...,198,2020-02-01,2020-01-02,교정기관의 민낮
3,안전/환경,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다.\r\r\n우리 일상에...,170,2020-02-01,2020-01-02,미세먼지 저감 대책
4,교통/건축/국토,저는 우선 아이셋의 부모입니다.\r\r\n식구가 많은편이고 아이들이 성장함에 따라 ...,"2,127",2020-02-01,2020-01-02,악질세입자 방지를 위한 세입자보호법을 재정해주세요.


## Preprocessing

In [5]:
# Before preprocessing
df.loc[1]['content']

'우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저평가된 시장이라고 합니다. 하지만 투자매력이 없다고도 합니다.이렇게 말하는 이유가 어디 있습니까? 바로 투자를 해도 수익을 기대하기 어렵다는 인식이 이미 널리 퍼져 있다는 것입니다.\r\n\r\r\n지금은 투자매력이 없어서 그렇지,..우리 나라 시중 부동 자금이 어마어마 한 것으로 알려진 것과, 외국 투자 자본 또한 실로 어마무지하게 많다는 것도 알고있습니다. 그러나 이 투자금이 주식시장으로 원활하게 순환이 안되고 있다는데 있습니다.\r\n\r\r\n정부는 유휴자금이 주식 시장으로 들어오게 분위기를 띄어줘야 하는데,... 그렇지 못한 현실이 안타깝다고 생각합니다.\r\r\n국가가 지원해 돈 들이기가 어려우면,정부에서 정서적인 말이라도 활성화를 위한 관심표명과 정책적으로 지원만 해도 시장분위기는 많이 좋아질 것이라 확신합니다. \r\n\r\r\n그래서 저는 정부에 다음과 같이 청원합니다.\r\r\n주식시장에 더 큰 충격 오기 전에 정부가 예방 조치를 할 수 있는데까지 노력해 주시기를 당부 드리면서,...\r\n\r\r\n첫째,주식시장 활성화와 부양에 대한 정부의 의지를 표명해 주십시오 \r\n\r\r\n둘째,  코스닥 종목은 공매도 제외시켜 주세요.\r\n\r\r\n공매도 제도를 아예 없애버리면 좋겠지만 제도를 살린다면 적용 시장에서 코스닥 종목은 제외 해 주어야 체력이 약한 코스닥 시장을 안정시킬 수 있다고 봅니다 \r\n\r\r\n셋째, 거래시장의 주식거래세를 더 낮춰 주세요 \r\n\r\r\n특히 개미(소액)투자에 동일한 요율로 부과하는 것은 너무나 불공정 하다고 봅니다.\r\n\r\r\n넷째, 중,장기 보유자에 대한 우대 차원에서 보유기간을 설정하여 주세요.\r\n\r\r\n이는 장기 보유자에 대한 혜택을 주어 초단타매매 등 시장 질서를 교란하는 투자자와 분리하는 제도를 도입해 시장 안정화에 기여해 주시길 간곡히 부탁드리면서 청원합니다.'

In [6]:
import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [7]:
# After preprocess
df.loc[1]['content']

'우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저평가된 시장이라고 합니다  하지만 투자매력이 없다고도 합니다 이렇게 말하는 이유가 어디 있습니까  바로 투자를 해도 수익을 기대하기 어렵다는 인식이 이미 널리 퍼져 있다는 것입니다      지금은 투자매력이 없어서 그렇지 우리 나라 시중 부동 자금이 어마어마 한 것으로 알려진 것과  외국 투자 자본 또한 실로 어마무지하게 많다는 것도 알고있습니다  그러나 이 투자금이 주식시장으로 원활하게 순환이 안되고 있다는데 있습니다      정부는 유휴자금이 주식 시장으로 들어오게 분위기를 띄어줘야 하는데  그렇지 못한 현실이 안타깝다고 생각합니다    국가가 지원해 돈 들이기가 어려우면 정부에서 정서적인 말이라도 활성화를 위한 관심표명과 정책적으로 지원만 해도 시장분위기는 많이 좋아질 것이라 확신합니다       그래서 저는 정부에 다음과 같이 청원합니다    주식시장에 더 큰 충격 오기 전에 정부가 예방 조치를 할 수 있는데까지 노력해 주시기를 당부 드리면서      첫째 주식시장 활성화와 부양에 대한 정부의 의지를 표명해 주십시오      둘째   코스닥 종목은 공매도 제외시켜 주세요      공매도 제도를 아예 없애버리면 좋겠지만 제도를 살린다면 적용 시장에서 코스닥 종목은 제외 해 주어야 체력이 약한 코스닥 시장을 안정시킬 수 있다고 봅니다      셋째  거래시장의 주식거래세를 더 낮춰 주세요      특히 개미 소액 투자에 동일한 요율로 부과하는 것은 너무나 불공정 하다고 봅니다      넷째  중 장기 보유자에 대한 우대 차원에서 보유기간을 설정하여 주세요      이는 장기 보유자에 대한 혜택을 주어 초단타매매 등 시장 질서를 교란하는 투자자와 분리하는 제도를 도입해 시장 안정화에 기여해 주시길 간곡히 부탁드리면서 청원합니다 '

# 2.3 토크나이징 및 변수 생성

In [8]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 37.4 MB/s eta 0:00:00


In [9]:
from konlpy.tag import Okt

okt = Okt()

df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

In [10]:
df['token_final'] = df.title_token + df.content_token

df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))

print(df.dtypes)

df['label'] = df['count'].apply(lambda x: 'Yes' if x>=1000 else 'No')

category         object
content          object
count             int64
end              object
start            object
title            object
title_token      object
content_token    object
token_final      object
dtype: object


In [11]:
df_drop = df[['token_final', 'label']]

In [12]:
df_drop.head()

,token_final,label
0,"[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...",No
1,"[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호, 우리, 나라, 코스피, 총...",No
2,"[교정, 기관, 의, 민낮, 일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이...",No
3,"[미세먼지, 저, 감, 대책, 미세먼지, 심각, 성은, 이제, 적극, 대안, 요구,...",No
4,"[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...",Yes


In [13]:
df_drop.to_csv('data/df_drop.csv', index = False, encoding = 'utf-8-sig')

## Word Embedding

In [14]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(
    df_drop['token_final'],
    sg = 1, # skip-gram
    vector_size = 100,
    window = 2,
    min_count = 1,
    workers = 4
)

print(embedding_model)

model_result = embedding_model.wv.most_similar("음주운전")
print(model_result)

Word2Vec<vocab=42642, vector_size=100, alpha=0.025>
[('음주', 0.8906262516975403), ('뺑소니', 0.8236576318740845), ('무면허', 0.8191040754318237), ('전과자', 0.7793022990226746), ('살인자', 0.7740102410316467), ('살인죄', 0.7527707815170288), ('강력범죄', 0.751740038394928), ('윤창', 0.7453432083129883), ('경범죄', 0.7433671355247498), ('승자', 0.7316267490386963)]


In [15]:
embedding_model.wv.save_word2vec_format('data/petitions_tokens_w2v')

In [16]:
from gensim.models import KeyedVectors

loaded_model = KeyedVectors.load_word2vec_format('data/petitions_tokens_w2v')

model_result = loaded_model.most_similar("음주운전")
print(model_result)

[('음주', 0.8906262516975403), ('뺑소니', 0.8236576318740845), ('무면허', 0.8191040754318237), ('전과자', 0.7793022990226746), ('살인자', 0.7740102410316467), ('살인죄', 0.7527707815170288), ('강력범죄', 0.751740038394928), ('윤창', 0.7453432083129883), ('경범죄', 0.7433671355247498), ('승자', 0.7316267490386963)]


## Training

In [17]:
from numpy.random import RandomState


rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

In [18]:
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import numericalize_tokens_from_iterator


# tokenizer = get_tokenizer("spacy")

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text


def token_gen(text):
    """
    Tokenizes each sentence in a given text and yields the resulting tokens.
    Args:
        text (list[str]): A list of sentences to tokenize.
    Yields:
        list[str]: The resulting tokens from each sentence.
    """
    for sent in text:
        tokens = tokenizer(sent)
        yield tokens


/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [19]:
import pandas as pd


df = pd.read_csv('data/train.csv')
df_valid = pd.read_csv('data/validation.csv')

In [20]:
df.head()

,token_final,label
0,"['택배', '대형', '마트', '배송', '기사', '의', '과로', '를',...",Yes
1,"['공도', '에', '국가', '균형발전', '을', '위', '한', '전철',...",No
2,"['부실', '공사', '로', '국민', '의', '안전', '을', '위협', ...",No
3,"['텔레', '그램', '성', '착취', '사건', '피해자', '분', '들',...",Yes
4,"['알콜', '솜', '가격', '을', '내려주세요', '형', '당뇨', '질환...",No


In [21]:
import torch
from torch.nn.utils.rnn import pad_sequence


vocab = build_vocab_from_iterator(token_gen(df['token_final']),specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])  ## to handel OOV problem

# numericalize tokens from iterator using vocab
sequence = numericalize_tokens_from_iterator(vocab=vocab.get_stoi(),iterator=token_gen(df['token_final']))

# create a list to store tokenized sequences
token_ids = []
for i in range(len(df)):
    x = list(next(sequence))
    token_ids.append(x)

# Pad the sequences to the same length along dimension 0
padded_text = pad_sequence([torch.tensor(x) for x in token_ids], batch_first=True, padding_value=0)

In [22]:
# restrict the length of every sequence
MAX_LENGTH = 100 # to restrict length every sequence
padded_text = padded_text[:,:MAX_LENGTH]

In [23]:
df['label'] = df['label'].replace({'Yes': 1, 'No': 0})

In [24]:
df_valid['label'] = df_valid['label'].replace({'Yes': 1, 'No': 0})

In [25]:
# label of the sentiment for train data
label = df['label'].to_list()
label= torch.tensor(label)

In [26]:
# X_train,y_train
X_train,y_train = padded_text,label


# numericalize tokens from iterator using vocab for df_valid
sequence_valid = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_gen(df_valid['token_final']))

# create a list to store tokenized sequences
valid_token_ids = []
for i in range(len(df_valid)):
    x = list(next(sequence_valid))
    valid_token_ids.append(x)

# Pad the sequences to the same length along dimension 0
padded_text_valid = pad_sequence([torch.tensor(x) for x in valid_token_ids], batch_first=True, padding_value=0)
# here look, <UNK> will be assign to 0 and padding_idx will be assign also 0

# restrict the length of every sequence upto MAX_LENGTH
padded_text_valid = padded_text_valid[:,:MAX_LENGTH]

label_valid = df_valid['label'].to_list()
label_valid = torch.tensor(label_valid)

# X_test,y_test
X_test, y_test = padded_text_valid, label_valid


# Determine the number of classes
num_classes = len(label.unique())

VOCAB_SIZE = len(vocab.get_stoi())
weights = torch.FloatTensor(loaded_model.vectors)

In [27]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class TextCNN(nn.Module):
    def __init__(self, w2v_weights, vocab_size, emb_dim, dim_channel, kernel_wins, num_class):
        super(TextCNN, self).__init__()

        self.vocab_size = vocab_size
        self.embed = nn.Embedding.from_pretrained(w2v_weights)  # Reference: <https://stackoverflow.com/a/49802495/9012940>

        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)

    def forward(self, x):

        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]

        fc_x = torch.cat(pool_x, dim=1)
        fc_x = fc_x.squeeze(-1)
        fc_x = self.dropout(fc_x)

        logit = self.fc(fc_x)

        return logit

In [35]:
def train(model, device, train_itr, optimizer):

    model.train()
    corrects, train_loss = 0.0,0

    for batch in train_itr:

        text, target = batch
        # text = torch.transpose(text, 0, 1)
        # target.data.sub_(1)
        text, target = text.to(device), target.to(device)

        optimizer.zero_grad()
        logit = model(text)
        # print(logit)
        # print(text)
        # print(target)

        loss = F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

In [29]:
def evaluate(model, device, itr):

    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:

        text = batch[0]
        target = batch[1]
        # text = torch.transpose(text, 0, 1)
        # target.data.sub_(1)
        text, target = text.to(device), target.to(device)

        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset)
    accuracy = 100.0 * corrects / len(itr.dataset)

    return test_loss, accuracy

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available() and not torch.cuda.is_available():
    device = torch.device("mps")


model = TextCNN(weights, VOCAB_SIZE, 100, 10, [3, 4, 5], 2).to(device)
print(model)

optimizer = optim.Adam(model.parameters(), lr=0.00005) #lr=0.001

TextCNN(
  (embed): Embedding(42642, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)


In [41]:
X_train,y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

zip = torch.utils.data.TensorDataset(X_train, y_train)
train_iter = torch.utils.data.DataLoader(zip, batch_size=16, shuffle=True)

In [42]:
X_train.shape, y_train.shape

(torch.Size([8705, 100]), torch.Size([8705]))

In [43]:
for i in train_iter:
    print(i)
    break

[tensor([[   11,   901,  1127,  ...,  3118,  8010,    99],
        [ 2054,   737,  1072,  ...,    66,   100,   247],
        [14842,   573,  3214,  ..., 24025,    19,    41],
        ...,
        [  223,    43,   833,  ...,   192,   714,     1],
        [    8,    86,  2431,  ...,    31,   200,  1578],
        [  248,    68,   142,  ...,   476,   381,   124]], device='cuda:0'), tensor([1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], device='cuda:0')]


In [46]:
best_test_acc = -1

EPOCHS = 30

for epoch in range(1, EPOCHS+1):
    X_train,y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    zip = torch.utils.data.TensorDataset(X_train, y_train)
    train_iter = torch.utils.data.DataLoader(zip, batch_size=16, shuffle=True)

    tr_loss, tr_acc = train(model, device, train_iter, optimizer)
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))

    zip_val = torch.utils.data.TensorDataset(X_test, y_test)
    val_iter = torch.utils.data.DataLoader(zip_val, batch_size=32, shuffle=True)

    val_loss, val_acc = evaluate(model, device, val_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))

    if val_acc > best_test_acc:
        best_test_acc = val_acc

        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")

    print('-----------------------------------------------------------------------------')

Train Epoch: 1 	 Loss: 0.043759050573308464 	 Accuracy: 52.41815185546875%
Valid Epoch: 1 	 Loss: 0.021431220613200876 	 Accuracy: 55.83639907836914%
model saves at 55.83639907836914 accuracy
-----------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.04296595614097503 	 Accuracy: 55.611717224121094%
Valid Epoch: 2 	 Loss: 0.02140745602767257 	 Accuracy: 55.83639907836914%
-----------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.042857254708655465 	 Accuracy: 55.57725524902344%
Valid Epoch: 3 	 Loss: 0.021391170811565483 	 Accuracy: 55.83639907836914%
-----------------------------------------------------------------------------
Train Epoch: 4 	 Loss: 0.04271595148708819 	 Accuracy: 56.484779357910156%
Valid Epoch: 4 	 Loss: 0.021340382230632445 	 Accuracy: 55.83639907836914%
-----------------------------------------------------------------------------
Train Epoch: 5 	 Loss: 0.04250917362797062 	 Ac